# Weather Retrieval

Using the free API from [Open Weather Map](https://openweathermap.org/), I would like to collect my local weather information.  WIth this information, I then want eventually to store it in a SQLite database to back-track weather patterns.  Certain weather changes are more difficult to keep track of, such as barometric pressure and wind speed.  These are usually aggregated for prior days, so storing them makes its easier to find interesting fun facts of local weather changes.  I'd also like to incorporate a Flask webpage for displaying or querying certain parameters.

<br> Altnerate City IDs include:
 - Austin: 4671654
 - Chicago: 4887398
 - New York: 5128581
 - Pittsburgh: 5206379
 - Tahoe: 5599665


In [ ]:
import os
import requests
import datetime
import sqlite3
import pprint as pp

In [ ]:
weather_api_key = os.getenv('weather_api_key')
city_id =  os.getenv('weather_api_my_city_id')

In [ ]:
f'https://api.openweathermap.org/data/2.5/weather?id={city_id}&APPID={weather_api_key}'

In [ ]:
r = requests.get(f'https://api.openweathermap.org/data/2.5/weather?id={city_id}&APPID={weather_api_key}')

## Scratch area
This area is a continuation of scratch. Some code was used to flatten the JSON to make it more manageable until a different method was used.  I am leaving this area for now in case I want to revisit the code.  None of these methods were used in the Database section

In [ ]:
pp.pprint(r.json())

In [ ]:
def flatten_json(json_object):
    output = {}
    
    def flatten(x, name=''):
        if isinstance(x, dict):
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) +'_')
                i += 1
        else:
            output[name[:-1]] = x
            
    flatten(json_object)
    return output

In [ ]:
weather_dict = flatten_json(r.json())

In [ ]:
pp.pprint(weather_dict)

In [ ]:
for key, value in r.json().items():
    if isinstance(value, dict):
        for k, v in value.items():
            print(f'\t{key} | {k}, {v}')
    elif isinstance(value, list):
        for i in range(len(value)):
            print(flatten_json(value[i]))
    else:
        print((key, value))

In [ ]:
weather_cols = {}
for key, val in weather_dict.items():
    if type(val) == float:
        weather_cols[key] = 'REAL'
    elif type(val) == str:
        weather_cols[key] = 'TEXT'
    else:
        weather_cols[key] = 'INT'
for key, val in weather_cols.items():
    print(key, val)

In [ ]:
weather_sqlite_cols = ''
for i, (key, val) in enumerate(weather_cols.items()):
    if i < len(weather_cols)-1:
        print(f'{key} {val}, ')
        weather_sqlite_cols += f'{key} {val}, '
    else:
        print(f'{key} {val}')
        weather_sqlite_cols += f'{key} {val}'
print(weather_sqlite_cols)

", ".join(weather_cols.values())

# SQLite
## Creation and Data Entry
Create a SQLite database using SQLalchemy.  Create classes to make it easier to automagically insert objects into the database as well as to ignore dictionary items that do not exist in a returned API request.

<br> I am using SQLite for this application.  A nifty SQLite viewer can be found at [DB Browser for SQLite](https://sqlitebrowser.org/)

In [ ]:
# where to store the database
path = os.path.abspath('.')
print(path)
weather_database = f'{path}\weather.sqlite'

In [ ]:
print(weather_database)

In [ ]:
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Sequence, Float, ForeignKey
from sqlalchemy.orm import sessionmaker
from dataclasses import dataclass

engine = create_engine(f'sqlite:///{weather_database}', echo=False)
Base = declarative_base()


In [ ]:
# Create Python data classes to make it easier/cleaner to put data 
# into SQLAlchemy objects.  Currently have a City and Weather Attribute class

@dataclass
class City(Base):
    __tablename__ = 'city'
    
    #id = Column(Integer, Sequence('city_id_seq'), primary_key=True)
    id = Column(Integer, primary_key=True)
    city_name = Column(String(50))
    country_name = Column(String(50))
    longitude = Column(Float)
    latitude = Column(Float)

@dataclass
class WeatherAttributes(Base):
    __tablename__ = 'weatherattributes'
    
    id = Column(Integer, Sequence('weather_id_seq'), primary_key=True)
    city_id = Column(Integer, ForeignKey("city.id"))
    
    dt = Column(Integer)
    sunrise_dt = Column(Integer)
    sunset_dt = Column(Integer)
    temp_current = Column(Float)
    temp_max = Column(Float)
    temp_min = Column(Float)
    humidity = Column(Integer)
    visibility = Column(Float)
    pressure = Column(Integer)
    wind_deg = Column(Integer)
    wind_spd = Column(Float)
    clouds = Column(Integer)
    snow_1h = Column(Float)
    snow_3h = Column(Float)
    rain_1h = Column(Float)
    rain_3h = Column(Float)
    
'''
# Weather Patterns class.  Can be a dictionary with 
# multiple lists inside the dictionary.  Saving this for another time.

@dataclass
class WeatherPatterns(Base):
    __tablename__ = 'weatherpatterns'
    
    weather_id = Column(Integer)
    weather_description = Column(String(50))
    weather_icon = Column(String(50))
    clouds = Column(Integer)

# System attributes proprietary to openweathermap.
# May not be necessary for me to store.
@dataclass
class System(Base):
    __tablename__ = 'system'
    sys_id = Column(Integer)
    sys_message = Column(String(50))
    sys_type = Column(Integer)
    code = Column(Integer)'''
    

In [ ]:
Base.metadata.create_all(engine)

In [ ]:
@dataclass
class insert_city:
    
    def __init__(self, weather_dict):
        city_name = weather_dict['name']
        id = weather_dict['id']
        country_name = weather_dict['sys']['country']
        longitude = weather_dict['coord']['lon']
        latitude = weather_dict['coord']['lat']
    
@dataclass
class insert_weather_event:
    
    def __init__(self, weather_dict):
        self.dt = weather_dict.get('dt')
        self.city_id = weather_dict.get('id')
        self.sunrise_dt = weather_dict.get('sys').get('sunrise')
        self.sunset_dt = weather_dict.get('sys').get('sunset')
        self.visibility = weather_dict.get('visibility')
        self.humidity = weather_dict.get('main').get('humidity')
        self.pressure = weather_dict.get('main').get('pressure')
        self.temp_current = weather_dict.get('main').get('temp')
        self.temp_max = weather_dict.get('main').get('temp_min')
        self.temp_min = weather_dict.get('main').get('temp_max')
        self.wind_deg = weather_dict.get('wind').get('deg')
        self.wind_spd = weather_dict.get('wind').get('speed')
        self.clouds = weather_dict.get('clouds').get('all')
        if isinstance(weather_dict.get('snow'), dict):
            self.snow_1h = weather_dict.get('snow').get('1h')
            self.snow_3h = weather_dict.get('snow').get('3h')
        if isinstance(weather_dict.get('rain'), dict):
            self.rain_1h = weather_dict.get('rain').get('1h')
            self.rain_3h = weather_dict.get('rain').get('3h')
            
'''     Need to figure out what to do with weather events
        Can be multiple nested lists in the weather dict
        self.weather_description = weather_dict.get('weather').get('description')
        self.weather_icon = weather_dict.get('weather').get('description')
        self.weather_id = weather_dict.get('weather').get('id')
        self.weather_main = weather_dict.get('weather').get('main')'''

In [ ]:
# Weather even JSON object to be passed in later variables
# Unique to the city and timestamp data was retrieved
weather_event = r.json()

In [ ]:
# Pass the weather_event JSON to the city or weather classes
c = insert_city(weather_event)
w = insert_weather_event(weather_event)
# Unpack the weather class into the weather database object
newEvent = WeatherAttributes(**w.__dict__)

In [ ]:
# Check if a city is added to the database
# If a city does not exist, add it to the database
Session = sessionmaker()
Session.configure(bind=engine) # once engine is available
session = Session()
city_already_added = session.query(City).filter_by(id=weather_event['id']).first()
if not city_already_added:
    newCity = City(**c.__dict__)
    session.add(newCity)
else:
    session.rollback()
session.commit()

In [ ]:
# Commit the latest weather event to the database
Session = sessionmaker()
Session.configure(bind=engine)
session = Session()
session.add(newEvent)
session.commit()

# Obsolete
Replaced this with an unpackable class.
<br>Allows for certain elements to activate in if statements

newCity = City(city_name = weather_event['name'],
               id = weather_event['id'],
               country_name = weather_event['sys']['country'],
               longitude = weather_event['coord']['lon'],
               latitude = weather_event['coord']['lat'])
               
newEvent = WeatherAttributes(
    dt = weather_event.get('dt'),
    city_id = weather_event.get('id'),
    sunrise_dt = weather_event.get('sys').get('sunrise'),
    sunset_dt = weather_event.get('sys').get('sunset'),
    visibility = weather_event.get('visibility'),
    humidity = weather_event.get('main').get('humidity'),
    pressure = weather_event.get('main').get('pressure'),
    temp_current = weather_event.get('main').get('temp'),
    temp_max = weather_event.get('main').get('temp_min'),
    temp_min = weather_event.get('main').get('temp_max'),
    #weather_description = weather_event.get('weather').get('description'),
    #weather_icon = weather_event.get('weather').get('description'),
    #weather_id = weather_event.get('weather').get('id'),
    #weather_main = weather_event.get('weather').get('main'),
    wind_deg = weather_event.get('wind').get('deg'),
    wind_spd = weather_event.get('wind').get('speed'),
    clouds = weather_event.get('clouds').get('all'),
    snow_1h = weather_event.get('snow').get('1h'),
    snow_3h = weather_event.get('snow').get('3h'),
    rain_1h = weather_event.get('rain').get('1h'),
    rain_3h = weather_event.get('rain').get('3h')
)